# Slackbot Example

In this notebook, you’ll see how to train BeepGPT on your Slack history in 15 minutes using only OpenAI’s API’s and open-source Python libraries - Data Science PhD not required.

We'll train BeepGPT in four steps:
1. Pull down historical messages
2. Build training examples
3. Convert our examples into a training dataset of prompt/completion pairs
4. Send our training data to OpenAI and create a fine-tuning job

In [ ]:
%pip install timestreams pandas pyarrow openai

In [1]:
from datetime import datetime, timedelta
import sparrow_py as kt
import pandas
import openai
import getpass
import pyarrow
import datetime

# Initialize Kaskada with a local execution context.
kt.init_session()

## Read Historical Messages

Historical slack messages can be exported by following the instructions in Slack's [Export your workspace data](https://slack.com/help/articles/201658943-Export-your-workspace-data) web page. We'll use these messages to teach BeepGPT about the members of your workspace.

In [2]:
# Load events from a Parquet file
# Use the "ts" column as the time associated with each row, 
# and the "channel" column as the entity associated with each row.
messages = kt.sources.Parquet(
    path = "./messages.parquet", 
    time_column_name = "ts_parsed", 
    key_column_name = "channel",
)

# View the first 5 events
messages.preview(5)

,_time,_subsort,_key_hash,_key,subtype,ts,ts_parsed,user,text,team,...,reactions,thread_ts,reply_count,reply_users_count,latest_reply,is_locked,subscribed,last_read,parent_user_id,channel
0,2023-07-25 19:42:13,5,15750806798332339587,general,message,1690314133.159559,2023-07-25 19:42:13,U05JQJJDJ6P,<@U05JQJJDJ6P> has joined the channel,None,...,None,None,NaN,NaN,None,None,None,None,None,general
1,2023-07-25 19:42:14,14,3094307063304068259,random,message,1690314133.550289,2023-07-25 19:42:14,U05JQJJDJ6P,<@U05JQJJDJ6P> has joined the channel,None,...,None,None,NaN,NaN,None,None,None,None,None,random
2,2023-07-25 19:44:27,0,2954779196800164886,demo,message,1690314266.912869,2023-07-25 19:44:27,U05JQJJDJ6P,<@U05JQJJDJ6P> has joined the channel,None,...,None,None,NaN,NaN,None,None,None,None,None,demo
3,2023-07-26 08:29:35,6,15750806798332339587,general,message,1690360175.262899,2023-07-26 08:29:35,U05JQJJDJ6P,old message 1,T05JA5XCR9D,...,None,None,NaN,NaN,None,None,None,None,None,general
4,2023-07-26 08:29:37,7,15750806798332339587,general,message,1690360176.582019,2023-07-26 08:29:37,U05JQJJDJ6P,old message 2,T05JA5XCR9D,...,None,None,NaN,NaN,None,None,None,None,None,general


## Build examples

Fine-tuning examples will teach the model the specific users who are interested in a given conversation. Each example consists of a "prompt" containing the state of a conversation at a point in time and a "completion" containing the users (if any) who were interested in the conversation. BeepGPT uses several ways to measure interest, for example, replying to a message, or adding an emoji reaction.

In [3]:
# Re-group messages by thread and/or channel
# Slack messages are delivered chronologically, so messages in threads
# may be interleaved with messages in the main channel.
messages = messages.with_key(kt.record({
        "channel": messages.col("channel"),
        "thread": messages.col("thread_ts"),
    }))

# Build the GPT input prompt by collecting relevant fields of recent messages
conversations = messages \
    .select("user", "ts", "text", "reactions") \
    .collect(max=20)


# Shift the prompt forward in time 5m to observe the effects of the conversation
shifted_conversations = conversations.shift_by(datetime.timedelta(seconds=1))

# Collect all the users who reacted to the conversation in the past 5m
# (the period of time the prompt was shifted across)
reaction_users = messages \
    .collect(window=kt.windows.Trailing(datetime.timedelta(seconds=1)), max=100) \
    .col("reactions").flatten() \
    .col("users").flatten()

# Collect all the users to posted messages in the past 5m
participating_users = messages \
    .collect(window=kt.windows.Trailing(datetime.timedelta(seconds=1)), max=100) \
    .col("user")

# Build a fine-tuning example mapping a conversation to the users who reacted to it
history = kt.record({
        "conversation": shifted_conversations, 
        "engaged_users": reaction_users.union(participating_users),
    }) \
    .filter(shifted_conversations.is_not_null())

history.preview(5)

,_time,_subsort,_key_hash,_key,conversation,engaged_users
0,2023-07-25 19:42:14,0,10930785253300487896,"{'channel': 'general', 'thread': None}","[{'ts': '1690314133.159559', 'user': 'U05JQJJD...",[U05JQJJDJ6P]
1,2023-07-25 19:42:15,1,3808896899993239160,"{'channel': 'random', 'thread': None}","[{'ts': '1690314133.550289', 'user': 'U05JQJJD...",[U05JQJJDJ6P]
2,2023-07-25 19:44:28,2,17093109913058365975,"{'channel': 'demo', 'thread': None}","[{'ts': '1690314266.912869', 'user': 'U05JQJJD...",[U05JQJJDJ6P]
3,2023-07-26 08:29:35,6,10930785253300487896,"{'channel': 'general', 'thread': None}","[{'ts': '1690314133.159559', 'user': 'U05JQJJD...",[U05JQJJDJ6P]
4,2023-07-26 08:29:36,3,10930785253300487896,"{'channel': 'general', 'thread': None}","[{'ts': '1690314133.159559', 'user': 'U05JQJJD...",[U05JQJJDJ6P]


## Create training dataset

To prepare our fine-tuning data for OpenAI, we'll use Scikit-Learn for preprocessing. This step ensures that each user is represented by a single "token", and that the conversation is formatted in a way that is easy for the model to learn

In [4]:
from sklearn import preprocessing
import numpy, json

# Extract examples from historical data
history_df = history.run().to_pandas().drop(["_time", "_subsort", "_key_hash", "_key"], axis=1)


# Encode user ID labels
le = preprocessing.LabelEncoder()
le.fit(history_df.engaged_users.explode())
with open('labels_.json', 'w') as f:
    json.dump(le.classes_.tolist(), f)


# Format for the OpenAI API
def format_prompt(conversation):
    return "start -> " + "\n\n".join([f' {msg["user"]} --> {msg["text"]} ' for msg in conversation]) + "\n\n###\n\n"
def format_completion(engaged_users):
    return " " + (" ".join(le.transform(engaged_users).astype(str)) if len(engaged_users) > 0 else "nil") + " end"
    
examples_df = pandas.DataFrame({
    "prompt": history_df.conversation.apply(format_prompt),
    "completion": history_df.engaged_users.apply(format_completion),
})

# Write examples to file
examples_df.to_json("examples.jsonl", orient='records', lines=True)

## Fine-tune a custom model

Finally, we'll send our fine-tuning examples to OpenAI to create a custom model.

In [ ]:
import openai
from openai import cli
from types import SimpleNamespace

# Initialize OpenAI
openai.api_key = getpass.getpass('OpenAI: API Key')

# Verifiy data format, split for training & validation, upload to OpenAI
args = SimpleNamespace(file='./examples.jsonl', quiet=True)
cli.FineTune.prepare_data(args)
training_id = cli.FineTune._get_or_upload('./examples_prepared_train.jsonl', True)

# Train a model using "davinci", the most advanced model available for fine-tuning
resp = openai.FineTune.create(
    training_file = training_id,
    model = "davinci",
    n_epochs = 2,
    learning_rate_multiplier = 0.02,
    suffix = "coversation_users"
)

# Fine-tuning can take awhile, so keep track of this ID
print(f'Fine-tuning model with job ID: "{resp["id"]}"')